##Star Schema — Dimension & Fact Tables

In [0]:
%sql
-- CREATE DIM CUSTOMER TABLE --

CREATE OR REPLACE TABLE amazon_sales.gold.gold_dim_customer AS
SELECT DISTINCT
    customer_id,
    customer_name,
    city,
    state,
    country
FROM amazon_sales.silver.amazon_sales_table;

In [0]:
%sql
-- CREATE DIM PRODUCT TABLE --

CREATE OR REPLACE TABLE amazon_sales.gold.gold_dim_product AS
SELECT DISTINCT
    product_id,
    product_name,
    category,
    sub_category,
    brand
FROM amazon_sales.silver.amazon_sales_table;

In [0]:
%sql

-- CREATE DIM DATE TABLE --

CREATE OR REPLACE TABLE amazon_sales.gold.gold_dim_date AS
SELECT DISTINCT
    order_date                              AS date,
    YEAR(order_date)                        AS year,
    MONTH(order_date)                       AS month,
    DATE_FORMAT(order_date, 'MMMM')        AS month_name,
    QUARTER(order_date)                     AS quarter,
    DAYOFWEEK(order_date)                   AS day_of_week,
    DATE_FORMAT(order_date, 'EEEE')        AS day_name,
    CASE WHEN DAYOFWEEK(order_date) 
         IN (1,7) THEN true 
         ELSE false END                     AS is_weekend
FROM amazon_sales.silver.amazon_sales_table
ORDER BY date;

In [0]:
%sql
-- CREATE DIM PAYMENT TABLE --

CREATE OR REPLACE TABLE amazon_sales.gold.gold_dim_payment AS
SELECT DISTINCT
    payment_method
FROM amazon_sales.silver.amazon_sales_table;

In [0]:
%sql
-- CREATE FACT SALES TABLE --

CREATE OR REPLACE TABLE amazon_sales.gold.gold_fact_sales AS
SELECT
    order_id,
    order_date,
    ship_date,
    delivery_date,
    customer_id,
    product_id,
    payment_method,
    order_status,
    quantity,
    unit_price,
    discount,
    shipping_cost,
    total_sales,
    total_sales_recalc,
    delivery_days,
    ship_days,
   _silver_timestamp
FROM amazon_sales.silver.amazon_sales_table;

##Aggregation Tables for Power BI


In [0]:
%sql

-- CREATE AGGREGATE TABLE FOR TOP BRANDS --


CREATE OR REPLACE TABLE amazon_sales.gold.gold_agg_top_brands AS
SELECT
    brand,
    category,
    sub_category,
    COUNT(order_id)                         AS total_orders,
    SUM(quantity)                           AS total_units_sold,
    ROUND(SUM(total_sales), 2)              AS total_revenue,
    ROUND(AVG(discount) * 100, 2)          AS avg_discount_pct,
    RANK() OVER (ORDER BY SUM(total_sales) DESC) AS revenue_rank
FROM amazon_sales.silver.amazon_sales_table
GROUP BY brand, category, sub_category
ORDER BY total_revenue DESC;

In [0]:
%sql

-- CREATE AGGREGATE TABLE FOR PAYMENT TREND TABLE --


CREATE OR REPLACE TABLE amazon_sales.gold.gold_agg_payment_trends AS
SELECT
    DATE_FORMAT(order_date, 'yyyy-MM')      AS year_month,
    payment_method,
    COUNT(order_id)                         AS total_orders,
    ROUND(SUM(total_sales), 2)              AS total_revenue,
    ROUND(AVG(total_sales), 2)              AS avg_order_value
FROM amazon_sales.silver.amazon_sales_table
GROUP BY DATE_FORMAT(order_date, 'yyyy-MM'), payment_method
ORDER BY year_month, payment_method;